## Notebook Contains

A Rag Agent using `Cewai` with short term memory

## Importing Libraries

In [1]:
import sqlite3
import datetime

import io
import contextlib
import ollama

import chromadb
from sentence_transformers import SentenceTransformer

from crewai import Agent, Task, Crew

In [2]:
MODEL_NAME = "ollama/llama3.2:latest"

## Creating Supervisor Agent with memory
This class will act as long term memory (persisted between runs).

In [3]:
# class SQLLightMemory:
#     def __init__(self, db_path = 'memory.db'):
#         self.conn = sqlite3.connect(db_path)
#         self.cur = self.conn.cursor(db_path)
#         self.cur.execute(""""
#                          CREATE TABLE IF NOT EXISTS memory(
#                             id INTEGER PRIMARY KEY AUTOINCREMENT,
#                             role TEXT,
#                             content TEXT,
#                             timestamp TEXT
#                          )"""
#                         )
#         self.conn.commit()
    
#     # Saving a new record
#     def save(self, role, content):
#         ts = datetime.datetime.now().isoformat()
#         self.cur.execute(""""INSERT INTO memory (role, content, timestamp) VALUES (?, ?, ?)""", (role, content, ts))
#         self.conn.commit()
    
#     # load most recent recors
#     def load_recent(self, limit = 10):
#         self.cur.execute(""""
#             SELECT role, content
#             FROM memory
#             ORDER BY id DESC LIMIT ?                 
#         """, (limit))
#         return self.cur.fetchall()

import sqlite3
import datetime

class SQLiteMemory:
    def __init__(self, db_path="memory.db"):
        self.conn = sqlite3.connect(db_path)
        self.cur = self.conn.cursor()
        self.cur.execute("""
            CREATE TABLE IF NOT EXISTS memory (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                role TEXT,
                content TEXT,
                timestamp TEXT
            )
        """)
        self.conn.commit()

    def save(self, role, content):
        ts = datetime.datetime.now().isoformat()
        self.cur.execute("INSERT INTO memory (role, content, timestamp) VALUES (?, ?, ?)", 
                         (role, content, ts))
        self.conn.commit()

    def load_recent(self, limit=10):
        self.cur.execute("SELECT role, content FROM memory ORDER BY id DESC LIMIT ?", (limit,))
        return self.cur.fetchall()


## Creating Python Coad Runner (Coading Agent)

In [4]:
def run_python_code(code: str) -> str:
    """Seafly run Python code and return output"""
    print(f"\t{code}")
    try:
        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            exec(code, {})
        return buffer.getvalue() or "Code executed successfully (no output)"
    except Exception as e:
        return f"Error running the code: {e}"

## Creating RAG Agent

In [5]:
class RAGChromaBD:
    def __init__(self, persist_dir = "./chroma_db"):
        self.client = chromadb.PersistentClient(path=persist_dir)
        self.model =  SentenceTransformer("all-MiniLM-L6-V2")

        # Create or Get connection
        self.collection = self.client.get_or_create_collection(
            name= "knowledge_base"
        )

    # Adding Document
    def add_documents(self, docs):
        embeddings = self.model.encode(docs).tolist()
        ids = [f"doc_{i}" for i in range(len(docs))]
        self.collection.add(documents=docs, embeddings= embeddings, ids=ids)

    # Query document
    def query(self, question, k=3):
        q_emb = self.model.encode([question]).tolist()
        results = self.collection.query(
            query_embeddings= q_emb,
            n_results= k
        )
        return results["documents"][0] if results["documents"] else []

## Creating Supervisor Agent

In [6]:
memory = SQLiteMemory()
ragdb = RAGChromaBD()

# preload docs into Chroma
ragdb.add_documents([
    "Python is a popular programming language.",
    "LangChain is a framework for building agents.",
    "Albert Einstein developed the theory of relativity."
])

supervisor = Agent(
    role= "supervisor",
    goal = "Route user queries to the right agent. Maintain long-term memory in SQLite.",
    backstory= (
        "You are the main controller. You chat with the human, "
        "decide if the query needs coding, knowledge retrieval, or casual chat."
    ),
    llm= MODEL_NAME
)

## Task Orchestration (Supervisor Chooses Path)

In [12]:
class SupervisorAgent:
    def __init__(self, memory: SQLiteMemory, model = MODEL_NAME):
        self.memory = memory
        self.model = model
        
    # method to deside which tools to use
    def decide_tool(self, query: str) -> str:
        context = self.memory.load_recent(limit=3)
        history = "\n".join([f"{r}: {c}" for r, c in context])

        system_prompt = """
            You are a Supervisor Agent that decides which specialized agent should handle a user query.
            Available agents:
            1. "coding_agent" → for math/code/executing Python
            2. "rag_agent" → for factual/knowledge/research questions
            3. "chat" → for general conversation or casual talk

            Rules:
            - Respond ONLY with one of: coding_agent, rag_agent, chat
            - Do not explain your choice
        """

        response = ollama.chat(
            model = self.model,
            messages = [
                {"role":"system", "content": system_prompt},
                {"role": "user", "content": f"Conversation history:\n{history}\n\nUser query: {query}"}
            ]
        )

        decision = response["message"]["content"].strip().lower()
        return decision

In [15]:
# Assume coding_agent and rag_agent are already defined

memory = SQLiteMemory()
supervisor = SupervisorAgent(memory=memory, model="llama3.2:latest")

def agentic_loop(user_query):
    decision = supervisor.decide_tool(user_query)

    print(f"Decision: {decision}")
    if decision == "coding_agent":
        print(f"user_query: {user_query}")
        result = run_python_code(user_query)
        memory.save("coding_agent", result)
    elif decision == "rag_agent":
        docs = ragdb.query(user_query)
        result = "\n".join(docs)
        memory.save("rag_agent", result)
    else:
        result = f"[Chat]: Sure! You said: {user_query}"
        memory.save("chat", result)

    return result


# 🔥 Test
# print(agentic_loop("What is the theory of relativity?"))  # → rag_agent
print(agentic_loop("12 * (3+2)"))                        # → coding_agent
print(agentic_loop("Hello, how are you?"))               # → chat


Decision: coding_agent
user_query: 12 * (3+2)
	12 * (3+2)
Code executed successfully (no output)
Decision: chat
[Chat]: Sure! You said: Hello, how are you?


In [13]:
while True:
    user_input = input("User: ")
    print(f"\n🙎‍♂️: {user_input}")
    if user_input.lower() in ["exit", "quit"]:
        break
    output = supervisor_router(user_input)
    print(f"🤖: {output}")


🙎‍♂️: print Hello world
🤖: As your supervisor, I noted your query: 'print Hello world'. Let's discuss!

🙎‍♂️: exit
